# Subquery

* A **subquery** is a query nested within another query.
* A **subquery** is also known as an inner query or nested query.
* A **subquery** can be useful for retrieving data that will be used by the main query as a condition for further data selection.

**SYNTAX**:
```postgresql
SELECT column_list
FROM table1
WHERE columnA operator (
    SELECT columnB 
    FROM table2
    WHERE condition
); 
```

**EXAMPLE**:
```postgresql
SELECT
  film_id
FROM
  film_category
  INNER JOIN category USING(category_id)
WHERE
  name = 'Action';
```

# Correlated Subquery

* In PostgreSQL, a **correlated subquery** is a subquery that references the columns from the outer query.
* Unlike a regular subquery, ***PostgreSQL evaluates the correlated subquery once for each row processed by the outer query***.
* Since PostgreSQL reevaluates the correlated subquery for every row in the outer query, this may lead to performance issues, especially when dealing with large datasets.
* A correlated subquery can be useful when you need to perform a query that depends on the values of the current row being processed.

**EXAMPLE**:
```postgresql
SELECT film_id, title, length, rating
FROM film f
WHERE length > (
    SELECT AVG(length)
    FROM film
    WHERE rating = f.rating
);
```

**How does it work?**

The outer query retrieves id, title, length, and rating from the **film** table that has the alias **f**.
```postgresql
SELECT film_id, title, length, rating
FROM film f
WHERE length > (...)
```

* For each row processed by the outer query, the correlated subquery calculates the average `length` of films that have the same `rating` as the current row (`f.rating`).
* The `WHERE` clause (`WHERE length > (...)`) checks if the length of the current film is greater than the average.

The correlated subquery calculates the average length for films with the same rating as the current row in the outer query:
```postgresql
SELECT AVG(length)
FROM film
WHERE rating = f.rating
```

* The `WHERE` clause ensures that the correlated subquery considers only films with the same rating as the current row in the outer query.
* The condition `rating = f.rating` creates the correlation.
* As a result, the outer query returns rows where the `length` of the film is greater than the average`length` for films with the same `rating`.

# ANY Operator

* The PostgreSQL `ANY` operator compares a value with a set of values returned by a subquery. 
* It is commonly used in combination with **comparison operators** such as `=, <, >, <=, >=, and <>`.

**SYNTAX**
```postgresql
expression operator ANY(subquery)
```
* **expression** is a value that you want to compare.
* **operator** is a comparison operator including `=, <, >, <=, >=, and <>`.
* **subquery** is a subquery that returns a set of values to compare against. It must return exactly one column.


The `ANY` operator returns `true` if the comparison returns `true` **for at least one of the values in the set**, and `false` otherwise.
* If the subquery returns an **empty set**, the result of `ANY` comparison is always `false`.
* Besides the **subquery**, you can use any construct that returns a set of values, such as an `ARRAY`.

> Note that `SOME` is a synonym for `ANY`, which means that you can use them interchangeably.

**EXAMPLE**:
```postgresql
SELECT * FROM employees
WHERE salary < ANY (
    SELECT salary FROM managers
);
```

# ALL Operator

The PostgreSQL `ALL` operator allows you to compare a value with all values in a set returned by a subquery.

**SYNTAX**
```postgresql
expression operator ALL(subquery)
```
* **expression** is a value that you want to compare.
* **operator** is a comparison operator including `=, <, >, <=, >=, and <>`.
* **subquery** is a subquery that returns a set of values to compare against. It must return exactly one column.


The `ALL` operator returns `true` if the comparison returns `true` **for all of the values in the set**, and `false` otherwise.
* If the subquery returns an **empty set**, the result of `ALL` comparison is always `true`.
* Besides the **subquery**, you can use any construct that returns a set of values, such as an `ARRAY`.

**EXAMPLE**:
```postgresql
SELECT * FROM employees
WHERE salary < ALL (
    SELECT salary FROM managers
) ORDER BY salary DESC;
```

# EXISTS operator

The `EXISTS` operator is a boolean operator that checks the existence of rows in a subquery.

**SYNTAX**
```postgresql
EXISTS (subquery)

NOT EXISTS (subquery)
```

Typically, you use the `EXISTS` operator in the `WHERE` clause of a `SELECT` statement:
```postgresql
SELECT column_list FROM table1
WHERE EXISTS(
    SELECT column_list FROM table2
    WHERE condition
);
```

* If the subquery returns at least one row, the `EXISTS` operator returns `true`. 
* If the subquery returns no row, the `EXISTS` returns `false`.
* Note that if the subquery returns `NULL`, the `EXISTS` operator returns `true`.
* The result of the `EXISTS` operator depends on whether any row is returned by the subquery, and not on the row contents. 
* Therefore, columns that appear in the **column_list** of the subquery are not important.

For this reason, the common coding convention is to write `EXISTS` in the following form:
```postgresql
SELECT column_list
FROM table1
WHERE EXISTS(
    SELECT 1 FROM table2
    WHERE condition
);
```

* To negate the `EXISTS` operator, you use the `NOT EXISTS` operator.
* The `NOT EXISTS` operator returns `true` if the subquery returns no row or `false` if the subquery returns at least one row.
* In practice, you often use the `EXISTS` operator in **conjunction with correlated subqueries**.

**EXAMPLE 1**: Use the `EXISTS` operator to check if the payment value is zero exists in the **payment** table.
```postgresql
SELECT EXISTS(
    SELECT 1 FROM payment
    WHERE amount = 0
);
```

**EXAMPLE 2**: Use the `EXISTS` operator to find customers who have paid at least one rental with an amount greater than 11.
```postgresql
SELECT
  first_name,
  last_name
FROM customer c
WHERE EXISTS (
    SELECT 1 FROM payment p
    WHERE
      p.customer_id = c.customer_id
      AND amount > 11
)
ORDER BY
  first_name,
  last_name; 
```

**EXAMPLE 3**: Use the NOT EXISTS operator to find customers who have not made any payment more than 11.
```postgresql
SELECT
  first_name,
  last_name
FROM customer c
WHERE NOT EXISTS (
    SELECT 1 FROM payment p
    WHERE
      p.customer_id = c.customer_id
      AND amount > 11
)
ORDER BY
  first_name,
  last_name; 
```

**`EXISTS` and `NULL` example**
The following example returns all rows from the customers table because the subquery in the `EXISTS` operator returns `NULL`:

```postgresql
SELECT
  first_name,
  last_name
FROM customer
WHERE EXISTS(
    SELECT NULL
)
ORDER BY
  first_name,
  last_name; 
```

**Output:**
```
first_name  |  last_name
-------------+--------------
 Aaron       | Selby
 Adam        | Gooch
 Adrian      | Clary
 Agnes       | Bishop
...`
```